In [34]:
# Importing libraries 
import boto3
import json
import numpy as np
import pandas as pd
import time

## Importing dataset.

In [35]:
original_data = pd.read_csv('C://Users//rohan//Desktop//Recommendation engine//ml-25m//ratings.csv')

In [36]:
original_data.head(10)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
5,1,1088,4.0,1147868495
6,1,1175,3.5,1147868826
7,1,1217,3.5,1147878326
8,1,1237,5.0,1147868839
9,1,1250,4.0,1147868414


In [37]:
#Validating if we have enough number of users
print("Unique Users:" + str(len(original_data['userId'].unique())))
print("Unique Movies:" + str(len(original_data['movieId'].unique())))
print("Number of Interactions" + str(len(original_data.index)))
# well above  movies >50 no of interactions>2500

Unique Users:162541
Unique Movies:59047
Number of Interactions25000095


# Preparing the dataset 

In [38]:
import time
from time import sleep 
from datetime import datetime
#we want to change the time stamp to something which is more readable 
arb_time_stamp= original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))  
## data is relatively modern

1147878050.0
2006-05-17 15:00:50


In [39]:
#Now personalizing the dataset for personalize 
cleaned_data = original_data.copy()
cleaned_data=cleaned_data[cleaned_data['rating'] >= 3]

In [40]:
watched_df=cleaned_data.copy()
watched_df=watched_df[watched_df['rating']>4]
watched_df=watched_df[['userId','movieId','timestamp']]
watched_df['EVENT_TYPE']='watch'
watched_df.head()

,userId,movieId,timestamp,EVENT_TYPE
0,1,296,1147880044,watch
2,1,307,1147868828,watch
3,1,665,1147878820,watch
8,1,1237,1147868839,watch
16,1,2351,1147877957,watch


In [41]:
clicked_df = cleaned_data.copy()
clicked_df=clicked_df[clicked_df['rating']>=3]
clicked_df=clicked_df[['userId','movieId','timestamp']]
clicked_df['EVENT_TYPE']='click'
clicked_df.head()

,userId,movieId,timestamp,EVENT_TYPE
0,1,296,1147880044,click
1,1,306,1147868817,click
2,1,307,1147868828,click
3,1,665,1147878820,click
4,1,899,1147868510,click


In [42]:
interactions_df = clicked_df.copy()
interactions_df=interactions_df.append(watched_df)
interactions_df.sort_values("timestamp",axis=0, ascending=True, inplace=True, na_position='last')

In [43]:
interactions_df.rename(columns = {'userId':'USER_ID','movieId':'ITEM_ID','timestamp':'TIMESTAMP'}, inplace=True)

In [44]:
interactions_filename="interactions.csv"
interactions_df.to_csv(('C://Users//rohan//Desktop//Recommendation engine//ml-25m'+"//"+interactions_filename), index=False, float_format='%.0f')

# Creating item data 

In [45]:
original_data = pd.read_csv('C://Users//rohan//Desktop//Recommendation engine//ml-25m//movies.csv')
original_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [46]:
# title is not really needed 
itemmetadata_df=original_data.copy()
itemmetadata_df=itemmetadata_df[['movieId','genres']]
itemmetadata_df.head()

,movieId,genres
0,1,Adventure|Animation|Children|Comedy|Fantasy
1,2,Adventure|Children|Fantasy
2,3,Comedy|Romance
3,4,Comedy|Drama|Romance
4,5,Comedy


In [47]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [48]:
itemmetadata_df.rename(columns={'genres':'GENRE', 'movieId':'ITEM_ID'}, inplace=True)

In [49]:
itemmetadata_filename= "item-meta.csv"
itemmetadata_df.to_csv(('C://Users//rohan//Desktop//Recommendation engine//ml-25m'+"//"+itemmetadata_filename), index=False, float_format='%.0f')